# cs 5044 final project experiment 1: time-series forecasting

In [1]:
%matplotlib inline
import math
import pandas as pd
from pandas.plotting import lag_plot
from IPython.display import display
from sklearn import preprocessing
from sklearn import linear_model, neighbors
from sklearn import tree
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

### read in data

In [2]:
df_index = pd.read_csv('sp500_index.csv')
df_stocks = pd.read_csv('sp500_stocks.csv')
df_companies = pd.read_csv('sp500_companies.csv')

### preprocessing 

In [3]:
display(df_index)

,Date,S&P500
0,2012-10-11,1432.84
1,2012-10-12,1428.59
2,2012-10-15,1440.13
3,2012-10-16,1454.92
4,2012-10-17,1460.91
...,...,...
2510,2022-10-04,3790.93
2511,2022-10-05,3783.28
2512,2022-10-06,3744.52
2513,2022-10-07,3639.66


In [5]:
copy = df_index.copy()

In [6]:
copy.drop('Date', axis=1, inplace=True)

In [7]:
copy

,S&P500
0,1432.84
1,1428.59
2,1440.13
3,1454.92
4,1460.91
...,...
2510,3790.93
2511,3783.28
2512,3744.52
2513,3639.66


In [8]:
index_copy = copy.loc[::-1]
index_copy.reset_index(inplace=True)
index_copy.drop("index", axis=1, inplace=True)
index_copy

,S&P500
0,3612.39
1,3639.66
2,3744.52
3,3783.28
4,3790.93
...,...
2510,1460.91
2511,1454.92
2512,1440.13
2513,1428.59


In [9]:
df_t1 = index_copy[1:].reset_index(drop=True)
df_t2 = index_copy[2:].reset_index(drop=True)
df_t3 = index_copy[3:].reset_index(drop=True)
df_t4 = index_copy[4:].reset_index(drop=True)
df_t5 = index_copy[5:].reset_index(drop=True)
df_t6 = index_copy[6:].reset_index(drop=True)
df_t7 = index_copy[7:].reset_index(drop=True)

In [10]:
df_t1.columns = ['S&P1']
df_t2.columns = ['S&P2']
df_t3.columns = ['S&P3']
df_t4.columns = ['S&P4']
df_t5.columns = ['S&P5']
df_t6.columns = ['S&P6']
df_t7.columns = ['S&P7']

dfs = [index_copy, df_t1, df_t2, df_t3, df_t4, df_t5, df_t6, df_t7]

index_df = pd.concat(dfs, axis=1)

index_df

,S&P500,S&P1,S&P2,S&P3,S&P4,S&P5,S&P6,S&P7
0,3612.39,3639.66,3744.52,3783.28,3790.93,3678.43,3585.62,3640.47
1,3639.66,3744.52,3783.28,3790.93,3678.43,3585.62,3640.47,3719.04
2,3744.52,3783.28,3790.93,3678.43,3585.62,3640.47,3719.04,3647.29
3,3783.28,3790.93,3678.43,3585.62,3640.47,3719.04,3647.29,3655.04
4,3790.93,3678.43,3585.62,3640.47,3719.04,3647.29,3655.04,3693.23
...,...,...,...,...,...,...,...,...
2510,1460.91,1454.92,1440.13,1428.59,1432.84,NaN,NaN,NaN
2511,1454.92,1440.13,1428.59,1432.84,NaN,NaN,NaN,NaN
2512,1440.13,1428.59,1432.84,NaN,NaN,NaN,NaN,NaN
2513,1428.59,1432.84,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
index_df.dropna(inplace=True)

### autoregression

In [12]:
target = index_df['S&P500']
x = index_df.drop('S&P500', axis=1)

In [13]:
autoreg = linear_model.LinearRegression()

In [18]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

RMSEscores = []
MADscores = []

for train_index, test_index in kf.split(index_df):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    autoreg.fit(X_train, y_train)
    y_pred = autoreg.predict(X_test)
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    MADscores.append(metrics.mean_absolute_error(y_test, y_pred))
    print("RMSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print("MAD= {}".format(metrics.mean_absolute_error(y_test, y_pred)))
    print()

print("average RMSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print("average MAD=  {}".format(np.mean(MADscores)))
print("std=  {}".format(np.std(MADscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(target[-25:].values)

RMSE= 33.93002023154452
MAD= 19.909349466174415

RMSE= 32.48030193730174
MAD= 20.290274451441693

RMSE= 31.16334284830958
MAD= 19.88042713785731

RMSE= 35.65053450614469
MAD= 21.612763491160255

RMSE= 28.338842699576997
MAD= 17.744119705238166

average RMSE=  32.3126084445755
std=  2.4858607360636995
average MAD=  19.887386850374366
std=  1.2436533135222823

predicted:
[1540.71237496 1494.61878195 1516.46462265 1500.7857277  1513.02299607
 1500.03481305 1504.55243203 1508.18784827 1502.54956443 1503.44089568
 1496.46315897 1481.28375908 1474.79352477 1421.80590735 1434.8272351
 1438.7932993  1431.52889994 1415.59692268 1391.30586694 1361.46687674
 1384.1389076  1383.43369    1430.15949715 1417.18636347 1433.69225308]
ground truth:
[1409.93 1398.94 1406.29 1409.15 1391.03 1387.81 1386.89 1359.88 1353.33
 1355.49 1374.53 1380.03 1379.85 1377.51 1394.53 1428.39 1417.26 1414.2
 1427.59 1412.16 1411.94 1412.97 1408.75 1413.11 1433.82]
